In [2]:
class IntervalNode:

    def __init__(self, interval):
        self.interval = interval
        self.max = interval[1]
        self.left = None
        self.right = None


class IntervalTree:

    def __init__(self, intervals):
        self.root = self.build_tree(intervals)

    def build_tree(self, intervals):

        if not intervals:
            return None

        intervals = sorted(intervals, key=lambda x: x[0])
        mid = len(intervals) // 2
        root = IntervalNode(intervals[mid])

        root.left = self.build_tree(intervals[:mid])
        root.right = self.build_tree(intervals[mid + 1:])

        root.max = max(root.interval[-1],
                       root.left.max if root.left else float('-inf'),
                       root.right.max if root.right else float('-inf'))
        
        return root

In [3]:
intervals = [(1, 5), (2, 8), (3, 7), (10, 15), (12, 20)]

tree = IntervalTree(intervals)

In [5]:
import bisect
from typing import Dict, List, Set, Tuple

from docling_core.types.doc import DocItemLabel, Size
from rtree import index

class Interval:
    """Helper class for sortable intervals."""

    def __init__(self, min_val: float, max_val: float, id: int):
        self.min_val = min_val
        self.max_val = max_val
        self.id = id

    def __lt__(self, other):
        if isinstance(other, Interval):
            return self.min_val < other.min_val
        return self.min_val < other


class IntervalTree:
    """Memory-efficient interval tree for 1D overlap queries."""

    def __init__(self):
        self.intervals: List[Interval] = []  # Sorted by min_val

    def insert(self, min_val: float, max_val: float, id: int):
        interval = Interval(min_val, max_val, id)
        bisect.insort(self.intervals, interval)

    def find_containing(self, point: float) -> Set[int]:
        """Find all intervals containing the point."""
        pos = bisect.bisect_left(self.intervals, point)
        result = set()

        # Check intervals starting before point
        for interval in reversed(self.intervals[:pos]):
            if interval.min_val <= point <= interval.max_val:
                result.add(interval.id)
            else:
                break

        # Check intervals starting at/after point
        for interval in self.intervals[pos:]:
            if point <= interval.max_val:
                if interval.min_val <= point:
                    result.add(interval.id)
            else:
                break

        return result

In [6]:
tree = IntervalTree()
tree.insert(1, 5, 101)
tree.insert(3, 7, 102)
tree.insert(6, 10, 103)
tree.insert(4, 15, 104)

In [7]:
tree.find_containing(9)

{103, 104}